<a href="https://colab.research.google.com/github/Tmccoy22/fintech_project_2/blob/terry/nfl_predictions_running.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn import svm
import tensorflow as tp
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [58]:
# Upload credit_card_transactions.csv to Colab
from google.colab import files

csv_file = files.upload()

# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
NFL_df = pd.read_csv("NFL_data.csv")

# Review the DataFrame
NFL_df.head()

Saving NFL_data.csv to NFL_data (6).csv


,Date,Home Team,Away Team,Home Score,Away Score,Overtime?,Playoff Game?,Neutral Venue?,Home Odds Open,Home Odds Min,...,Total Score Close,Total Score Over Open,Total Score Over Min,Total Score Over Max,Total Score Over Close,Total Score Under Open,Total Score Under Min,Total Score Under Max,Total Score Under Close,Notes
0,2023-02-12,Philadelphia Eagles,Kansas City Chiefs,35,38,NaN,Y,Y,1.83,1.74,...,51.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,NaN
1,2023-01-29,Kansas City Chiefs,Cincinnati Bengals,23,20,NaN,Y,NaN,1.87,1.77,...,48.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,NaN
2,2023-01-29,Philadelphia Eagles,San Francisco 49ers,31,7,NaN,Y,NaN,1.77,1.65,...,45.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,NaN
3,2023-01-22,San Francisco 49ers,Dallas Cowboys,19,12,NaN,Y,NaN,1.50,1.48,...,46.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,NaN
4,2023-01-22,Buffalo Bills,Cincinnati Bengals,10,27,NaN,Y,NaN,1.52,1.38,...,48.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,NaN


In [59]:
#Drop Unused columns
NFL_odds_df = NFL_df.drop(columns = ["Overtime?", "Playoff Game?", "Neutral Venue?", "Notes", "Total Score Open", "Total Score Min", "Total Score Close",
               "Total Score Over Open","Total Score Over Min", "Total Score Over Max", "Total Score Over Close", "Total Score Under Open", 
               "Total Score Under Min" , "Total Score Under Max", "Total Score Under Close", "Total Score Max", "Date"])

In [60]:
NFL_odds_df.dtypes

Home Team                object
Away Team                object
Home Score                int64
Away Score                int64
Home Odds Open          float64
Home Odds Min           float64
Home Odds Max           float64
Home Odds Close         float64
Away Odds Open          float64
Away Odds Min           float64
Away Odds Max           float64
Away Odds Close         float64
Home Line Open          float64
Home Line Min           float64
Home Line Max           float64
Home Line Close         float64
Away Line Open          float64
Away Line Min           float64
Away Line Max           float64
Away Line Close         float64
Home Line Odds Open     float64
Home Line Odds Min      float64
Home Line Odds Max      float64
Home Line Odds Close    float64
Away Line Odds Open     float64
Away Line Odds Min      float64
Away Line Odds Max      float64
Away Line Odds Close    float64
dtype: object

In [61]:
#NFL_odds_df = NFL_odds_df.dropna()

In [62]:
NFL_odds_df['Home Team'] = NFL_odds_df['Home Team'].fillna(int(0), inplace= True)

In [63]:
NFL_odds_df['Away Team'] = NFL_odds_df['Away Team'].fillna(int(1), inplace = True)

In [64]:
NFL_odds_df['Home Team'] = NFL_odds_df['Home Team'].fillna(int(0))

In [65]:
NFL_odds_df['Away Team'] = NFL_odds_df['Away Team'].fillna(int(1),)

In [66]:
NFL_odds_df.head()

,Home Team,Away Team,Home Score,Away Score,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,...,Away Line Max,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close
0,0,1,35,38,1.83,1.74,1.83,1.83,2.00,2.00,...,2.5,1.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
1,0,1,23,20,1.87,1.77,2.20,1.77,1.95,1.71,...,2.0,2.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
2,0,1,31,7,1.77,1.65,1.77,1.65,2.10,2.10,...,3.0,3.0,1.87,2.00,1.87,2.00,1.95,1.95,1.83,1.83
3,0,1,19,12,1.50,1.48,1.54,1.50,2.70,2.65,...,4.0,3.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
4,0,1,10,27,1.52,1.38,1.52,1.38,2.67,2.67,...,6.0,6.0,1.95,1.91,1.95,1.91,1.87,1.87,1.91,1.91


In [11]:
#winner = []
#for i, row in NFL_odds_df.iterrows():
    #if row['Home Score'] > row['Away Score']:
     #   winner.append(row['Home Team'])
    #elif row['Away Score'] > row['Home Score']:
     #   winner.append(row['Away Team'])
    # Add the following line to skip appending a team name for tie games
    # and only append team names for games with a clear winner:
    #elif row['Home Score'] == row['Away Score']:
     #   continue

In [68]:
winner = []
for i, row in NFL_odds_df.iterrows():
    if row['Home Score'] > row['Away Score']:
        winner.append(row['Home Team'])
    elif row['Away Score'] > row['Home Score']:
        winner.append(row['Away Team'])
    else:
        winner.append(pd.np.nan)

<ipython-input-68-6bd6564e0bc0>:8: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  winner.append(pd.np.nan)


In [69]:
NFL_odds_df['winner'] = winner

In [70]:
NFL_odds_df.head()

,Home Team,Away Team,Home Score,Away Score,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,...,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close,winner
0,0,1,35,38,1.83,1.74,1.83,1.83,2.00,2.00,...,1.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.0
1,0,1,23,20,1.87,1.77,2.20,1.77,1.95,1.71,...,2.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,0.0
2,0,1,31,7,1.77,1.65,1.77,1.65,2.10,2.10,...,3.0,1.87,2.00,1.87,2.00,1.95,1.95,1.83,1.83,0.0
3,0,1,19,12,1.50,1.48,1.54,1.50,2.70,2.65,...,3.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,0.0
4,0,1,10,27,1.52,1.38,1.52,1.38,2.67,2.67,...,6.0,1.95,1.91,1.95,1.91,1.87,1.87,1.91,1.91,1.0


In [71]:
NFL_odds_df = NFL_odds_df.dropna()

In [72]:
# NFL_odds_filtered = NFL_odds_df[NFL_odds_df["winner"]!='Tie']

In [73]:
#NFL_odds_filtered

In [74]:
NFL_odds_df

,Home Team,Away Team,Home Score,Away Score,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,...,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close,winner
0,0,1,35,38,1.83,1.74,1.83,1.83,2.00,2.00,...,1.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.0
1,0,1,23,20,1.87,1.77,2.20,1.77,1.95,1.71,...,2.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,0.0
2,0,1,31,7,1.77,1.65,1.77,1.65,2.10,2.10,...,3.0,1.87,2.00,1.87,2.00,1.95,1.95,1.83,1.83,0.0
3,0,1,19,12,1.50,1.48,1.54,1.50,2.70,2.65,...,3.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,0.0
4,0,1,10,27,1.52,1.38,1.52,1.38,2.67,2.67,...,6.0,1.95,1.91,1.95,1.91,1.87,1.87,1.91,1.91,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2435,0,1,19,14,1.43,1.34,1.43,1.36,3.10,3.05,...,6.5,1.95,1.91,1.97,1.93,1.95,1.93,2.00,1.98,0.0
2436,0,1,34,17,1.20,1.18,1.21,1.19,5.23,5.08,...,10.0,1.91,1.93,2.02,2.02,2.00,1.88,1.98,1.88,0.0
2437,0,1,30,27,1.37,1.32,1.42,1.41,3.39,3.14,...,5.5,1.92,1.97,1.95,1.93,1.99,1.95,1.93,1.97,0.0
2438,0,1,6,34,1.50,1.50,1.73,1.65,2.81,2.24,...,3.0,1.95,1.95,1.88,2.02,1.95,2.02,1.95,1.88,1.0


In [75]:
NFL_odds_df.dropna()

,Home Team,Away Team,Home Score,Away Score,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,...,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close,winner
0,0,1,35,38,1.83,1.74,1.83,1.83,2.00,2.00,...,1.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.0
1,0,1,23,20,1.87,1.77,2.20,1.77,1.95,1.71,...,2.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,0.0
2,0,1,31,7,1.77,1.65,1.77,1.65,2.10,2.10,...,3.0,1.87,2.00,1.87,2.00,1.95,1.95,1.83,1.83,0.0
3,0,1,19,12,1.50,1.48,1.54,1.50,2.70,2.65,...,3.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,0.0
4,0,1,10,27,1.52,1.38,1.52,1.38,2.67,2.67,...,6.0,1.95,1.91,1.95,1.91,1.87,1.87,1.91,1.91,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2435,0,1,19,14,1.43,1.34,1.43,1.36,3.10,3.05,...,6.5,1.95,1.91,1.97,1.93,1.95,1.93,2.00,1.98,0.0
2436,0,1,34,17,1.20,1.18,1.21,1.19,5.23,5.08,...,10.0,1.91,1.93,2.02,2.02,2.00,1.88,1.98,1.88,0.0
2437,0,1,30,27,1.37,1.32,1.42,1.41,3.39,3.14,...,5.5,1.92,1.97,1.95,1.93,1.99,1.95,1.93,1.97,0.0
2438,0,1,6,34,1.50,1.50,1.73,1.65,2.81,2.24,...,3.0,1.95,1.95,1.88,2.02,1.95,2.02,1.95,1.88,1.0


In [76]:
data_df = NFL_odds_df.drop(columns=['Home Score', 'Away Score'])

In [77]:
data_df.head()

,Home Team,Away Team,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,Away Odds Max,Away Odds Close,...,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close,winner
0,0,1,1.83,1.74,1.83,1.83,2.00,2.00,2.15,2.00,...,1.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.0
1,0,1,1.87,1.77,2.20,1.77,1.95,1.71,2.10,2.10,...,2.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,0.0
2,0,1,1.77,1.65,1.77,1.65,2.10,2.10,2.35,2.35,...,3.0,1.87,2.00,1.87,2.00,1.95,1.95,1.83,1.83,0.0
3,0,1,1.50,1.48,1.54,1.50,2.70,2.65,2.75,2.70,...,3.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,0.0
4,0,1,1.52,1.38,1.52,1.38,2.67,2.67,3.15,3.15,...,6.0,1.95,1.91,1.95,1.91,1.87,1.87,1.91,1.91,1.0


In [78]:
y = data_df['winner']

In [79]:
y.head()

0    1.0
1    0.0
2    0.0
3    0.0
4    1.0
Name: winner, dtype: float64

In [80]:
X = data_df.drop(columns='winner')

In [81]:
X.head()

,Home Team,Away Team,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,Away Odds Max,Away Odds Close,...,Away Line Max,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close
0,0,1,1.83,1.74,1.83,1.83,2.00,2.00,2.15,2.00,...,2.5,1.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
1,0,1,1.87,1.77,2.20,1.77,1.95,1.71,2.10,2.10,...,2.0,2.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
2,0,1,1.77,1.65,1.77,1.65,2.10,2.10,2.35,2.35,...,3.0,3.0,1.87,2.00,1.87,2.00,1.95,1.95,1.83,1.83
3,0,1,1.50,1.48,1.54,1.50,2.70,2.65,2.75,2.70,...,4.0,3.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
4,0,1,1.52,1.38,1.52,1.38,2.67,2.67,3.15,3.15,...,6.0,6.0,1.95,1.91,1.95,1.91,1.87,1.87,1.91,1.91


In [82]:
X.tail()

,Home Team,Away Team,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,Away Odds Max,Away Odds Close,...,Away Line Max,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close
2435,0,1,1.43,1.34,1.43,1.36,3.10,3.05,3.56,3.46,...,6.5,6.5,1.95,1.91,1.97,1.93,1.95,1.93,2.00,1.98
2436,0,1,1.20,1.18,1.21,1.19,5.23,5.08,5.54,5.35,...,10.5,10.0,1.91,1.93,2.02,2.02,2.00,1.88,1.98,1.88
2437,0,1,1.37,1.32,1.42,1.41,3.39,3.14,3.73,3.16,...,7.0,5.5,1.92,1.97,1.95,1.93,1.99,1.95,1.93,1.97
2438,0,1,1.50,1.50,1.73,1.65,2.81,2.24,2.81,2.40,...,4.0,3.0,1.95,1.95,1.88,2.02,1.95,2.02,1.95,1.88
2439,0,1,1.45,1.38,1.47,1.43,2.99,2.91,3.32,3.05,...,6.0,4.5,1.97,1.93,2.00,1.93,1.93,1.91,1.98,1.98


In [83]:
print(X.isna().any().any())

False


In [84]:
print(y.isna().any().any())

False


In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [86]:
scaler = StandardScaler()

In [87]:
X_scaler = scaler.fit(X_train)

In [31]:
# Do we need to drop home/away scores? This is obviously the strongest predictor of who wins. Will model just focus on this?
# TensorFlow doesnt work in Jupyter lab?

In [88]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [33]:
#X_train= np.asarray(X_train).astype(np.float32)
#y_train = np.asarray(y_train).astype(np.float32)

In [34]:
#y_train = np.array(y_train)

In [89]:
number_input_features = len(X_train.iloc[0])

In [90]:
number_input_features

26

In [91]:
number_output_neurons = (1)

In [92]:
hidden_nodes_layer1 =  (number_input_features + 1) // 2 

In [93]:
hidden_nodes_layer1

13

In [94]:
hidden_nodes_layer2 =  (hidden_nodes_layer1 + 1) // 2

In [95]:
hidden_nodes_layer2

7

In [96]:
nn = Sequential()

In [97]:
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

In [98]:
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

In [99]:
nn.add(Dense(units=number_output_neurons, activation="sigmoid"))

In [100]:
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 13)                351       
                                                                 
 dense_4 (Dense)             (None, 7)                 98        
                                                                 
 dense_5 (Dense)             (None, 1)                 8         
                                                                 
Total params: 457
Trainable params: 457
Non-trainable params: 0
_________________________________________________________________


In [101]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [102]:
nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
57/57 [==============================] - 1s 2ms/step - loss: 0.6761 - accuracy: 0.6213
Epoch 2/50
57/57 [==============================] - 0s 2ms/step - loss: 0.6332 - accuracy: 0.6619
Epoch 3/50
57/57 [==============================] - 0s 2ms/step - loss: 0.6144 - accuracy: 0.6663
Epoch 4/50
57/57 [==============================] - 0s 2ms/step - loss: 0.6096 - accuracy: 0.6647
Epoch 5/50
57/57 [==============================] - 0s 2ms/step - loss: 0.6076 - accuracy: 0.6647
Epoch 6/50
57/57 [==============================] - 0s 2ms/step - loss: 0.6069 - accuracy: 0.6647
Epoch 7/50
57/57 [==============================] - 0s 2ms/step - loss: 0.6065 - accuracy: 0.6647
Epoch 8/50
57/57 [==============================] - 0s 2ms/step - loss: 0.6061 - accuracy: 0.6652
Epoch 9/50
57/57 [==============================] - 0s 2ms/step - loss: 0.6056 - accuracy: 0.6674
Epoch 10/50
57/57 [==============================] - 0s 2ms/step - loss: 0.6050 - accuracy: 0.6679
Epoch 11/50
57/57 [

In [103]:
X_train_scaled

array([[ 0.        ,  0.        , -0.19467395, ...,  0.16209631,
         0.78173746,  0.06346761],
       [ 0.        ,  0.        ,  0.33942459, ..., -1.56953581,
         0.28441705,  0.23302479],
       [ 0.        ,  0.        , -0.15107407, ...,  0.93171059,
         0.28441705,  1.75903934],
       ...,
       [ 0.        ,  0.        , -0.48897314, ..., -0.9923251 ,
         0.28441705, -1.1234326 ],
       [ 0.        ,  0.        , -0.4671732 , ..., -0.9923251 ,
        -0.5444503 , -0.44520391],
       [ 0.        ,  0.        , -0.9794718 , ...,  0.73930702,
        -0.5444503 , -0.44520391]])

In [104]:
y_train

2377    1.0
1439    0.0
1636    1.0
585     1.0
710     0.0
       ... 
964     0.0
909     1.0
1101    0.0
236     1.0
1065    0.0
Name: winner, Length: 1822, dtype: float64

In [ ]:
# Does the data need to be same format to work? Cannot combine array and list for model fit?

In [ ]:
# You need to go back and drop all NaNs at the very beginning of your dataset then rerun the code